<a href="https://colab.research.google.com/github/suchitbhayani/psychtweets/blob/main/model_training_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertModel
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [ ]:
from google.colab import files
uploaded = files.upload()

# Further Preprocessing

In [2]:
data = pd.read_csv('./data/cleaned.csv').drop(columns=['Unnamed: 0'])
data

FileNotFoundError: [Errno 2] No such file or directory: './data/cleaned.csv'

In [ ]:
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data

In [ ]:
train_text, test_text, train_labels, test_labels = train_test_split(data['cleaned text'].tolist(), data['label'].tolist(), test_size=0.2, random_state=5)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(text=train_text, truncation=True, padding=True)
test_encodings = tokenizer(text=test_text, truncation=True, padding=True)

# Dataset Class

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class MBTIDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    # returns a sample from dataset based on given idx
    def __getitem__(self, idx):
        item_dct = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item_dct['labels'] = torch.tensor(self.labels[idx])
        return item_dct

In [ ]:
train_data = MBTIDataset(train_encodings, train_labels)
test_data = MBTIDataset(test_encodings, test_labels)

In [ ]:
train_data[0]

# Building the Neural Network

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
hidden_layer_size = 64

In [ ]:
class MBTIPredictor(nn.Module):
    def __init__(self):
        super(MBTIPredictor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, 16)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.linear_relu_stack(outputs[1])
        return logits

In [ ]:
model = MBTIPredictor().to(device)
print(model)

# Model Training

https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [ ]:
learning_rate = 1e-4
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size = 16
epochs = 3

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, batch_data in enumerate(dataloader):
        input_ids = batch_data['input_ids'].to(device)
        attention_mask = batch_data['attention_mask'].to(device)
        labels = batch_data['labels'].to(device)

        # Compute prediction and loss
        pred = model(input_ids, attention_mask)
        loss = loss_fn(pred, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss_value = loss.item()
            current = batch * len(input_ids)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn, device):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch_data in dataloader:
            input_ids = batch_data['input_ids'].to(device)
            attention_mask = batch_data['attention_mask'].to(device)
            labels = batch_data['labels'].to(device)

            pred = model(input_ids, attention_mask)
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%\n")

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model = MBTIPredictor().to(device)

# Training loop
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loop(val_loader, model, loss_fn, device)

print("Done!")